In [ ]:
import pandas as pd


In [ ]:
# Read the table using Delta Lake with delimiter "," 
indo_spark_df = spark.read.format("csv").option("delimiter", ",").table("hive_metastore.default.tpa_indo_cleaned_all_digi_challenge")

indo_pd_df = indo_spark_df.toPandas()

# Read the table using Delta Lake with delimiter "," 
uk_spark_df = spark.read.format("csv").option("delimiter", ",").table("hive_metastore.default.tpa_uk_cleaned_all_digi_challenge")

uk_pd_df = uk_spark_df.toPandas()


In [ ]:
print(indo_pd_df.columns)

Index(['Customer', 'PromoIDText', 'PromoGroup', 'WeekSkID', 'InStoreStartWeek',
       'InStoreEndWeek', 'PromoFlag', 'PromotionStatus', 'InstoreStartDate',
       'InstoreEndDate', 'ShipmentStartDate', 'ShipmentEndDate',
       'PromoMechanic', 'PromoShopperMechanic', 'Level1Name_PPH', 'Category',
       'Level4Name_PPH', 'Brand', 'Level6Name_PPH', 'CPG', 'SPF',
       'ProductNameSku_PPH', 'fundtype', 'ListPrice',
       'PlannedPromoSalesValueSellIn', 'PlannedPromoSalesVolumeSellIn',
       'PlannedNetPromoGSVSellIn', 'PlannedNetPromoNIVSellIn',
       'PlannedTTSOnSpend', 'PlannedNetPromoTOSellIn', 'PlannedTTSOffSpend',
       'PlannedNetPromoGrossProfitsSellIn', 'PlannedNetPromoCOGSSellIn',
       'PlannedNetPromoPBOSellIn', 'PlannedBaselineValue',
       'PlannedBaselineVolume', 'PlannedBaseGSVSellIn',
       'PlannedBaseTTSOnSpend', 'PlannedBaseNIVSellIn',
       'PlannedBaseTTSOffSpend', 'PlannedBaseTOSellIn',
       'PlannedBaseGrossProfitsSellIn', 'PlannedBaseCOGSSellIn',
   

In [ ]:
print(uk_pd_df.columns)

Index(['Level8Code', 'Customer', 'PromoIDText', 'PromoFlag', 'TUEAN',
       'WeekSkID', 'InStoreStartWeek', 'InStoreEndWeek', 'PromotionStatus',
       'InstoreStartDate', 'InstoreEndDate', 'ShipmentStartDate',
       'ShipmentEndDate', 'PromoFeature', 'PromoMechanic',
       'PromoShopperMechanic', 'DivisionName_VG', 'CategoryName_VG',
       'SegmentName_VG', 'FormName_VG', 'EBFName_VG', 'Brand_VG', 'SPF',
       'SPFVName_VG', 'ListPrice', 'PlannedPromoSalesVolumeSellIn',
       'PlannedNetPromoGSVSellIn', 'PlannedTTSOnSpend',
       'PlannedNetPromoNIVSellIn', 'PlannedTTSOffSpend',
       'PlannedNetPromoTOSellIn', 'PlannedNetPromoGrossProfitsSellIn',
       'PlannedNetPromoCOGSSellIn', 'PlannedBaselineVolume',
       'PlannedBaseGSVSellIn', 'PlannedBaseTTSOnSpend', 'PlannedBaseNIVSellIn',
       'PlannedBaseTTSOffSpend', 'PlannedBaseTOSellIn',
       'PlannedBaseGrossProfitsSellIn', 'PlannedBaseCOGSSellIn',
       'PlannedEventSpend', 'ActualPromoSalesVolumeSellOut',
       'Actu

##Indonesia Model

In [ ]:
distinct_indo_df = indo_pd_df.drop_duplicates(["PromoIDText", "ProductNameSku_PPH"])

In [ ]:
# one-hot encode the PromoMechanic
# encoded_indo_df = pd.get_dummies(distinct_indo_df, columns=["PromoMechanic"], dtype=int)

In [ ]:
import torch
import torch.nn as nn

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # Define the layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 16 * 16, 10)  # Adjust the output size based on your task

    def forward(self, x):
        # Forward pass through the layers
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.fc1(x)
        return x

# Instantiate the model
model = SimpleCNN()

# Print the architecture
print(model)


SimpleCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4096, out_features=10, bias=True)
)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define the TCN block
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()

class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = nn.Conv1d(n_inputs, n_outputs, kernel_size,
                               stride=stride, padding=padding, dilation=dilation)
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = nn.Conv1d(n_outputs, n_outputs, kernel_size,
                               stride=stride, padding=padding, dilation=dilation)
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)

class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

# Define the model
class SalesForecastingModel(nn.Module):
    def __init__(self, input_channels, tcn_channels, output_size, kernel_size=2, dropout=0.2):
        super(SalesForecastingModel, self).__init__()
        self.tcn = TemporalConvNet(input_channels, tcn_channels, kernel_size, dropout)
        self.linear = nn.Linear(tcn_channels[-1], output_size)

    def forward(self, x):
        y1 = self.tcn(x)
        y2 = self.linear(y1[:, :, -1])
        return y2


# Number of planned features per week
input_channels = 5
# 2000 unique inputs isn't a large dataset, so we keep the model relatively small to avoid overfitting
tcn_channels = [16, 32, 64]
output_size = 5

model = SalesForecastingModel(input_channels, tcn_channels, output_size)
print(model)


def train():
    # Example training loop
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.002)

    # Random dataset for demonstration purposes
    # Let's assume X_train is of shape (batch_size, sequence_length, num_features)
    # and y_train is of shape (batch_size, output_size)
    X_train = np.random.rand(10, 5, 5).astype(np.float32)  # Batch of 10, 5 weeks data, 5 features
    y_train = np.random.rand(10, 5).astype(np.float32)     # Batch of 10, 5 predicted weeks

    X_train = torch.from_numpy(X_train).permute(0, 2, 1)   # Permute to (batch_size, num_features, sequence_length)
    y_train = torch.from_numpy(y_train)

    # Simple training loop
    num_epochs = 50
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

SalesForecastingModel(
  (tcn): TemporalConvNet(
    (network): Sequential(
      (0): TemporalBlock(
        (conv1): Conv1d(5, 16, kernel_size=(2,), stride=(1,), padding=(1,))
        (chomp1): Chomp1d()
        (relu1): ReLU()
        (dropout1): Dropout(p=0.2, inplace=False)
        (conv2): Conv1d(16, 16, kernel_size=(2,), stride=(1,), padding=(1,))
        (chomp2): Chomp1d()
        (relu2): ReLU()
        (dropout2): Dropout(p=0.2, inplace=False)
        (net): Sequential(
          (0): Conv1d(5, 16, kernel_size=(2,), stride=(1,), padding=(1,))
          (1): Chomp1d()
          (2): ReLU()
          (3): Dropout(p=0.2, inplace=False)
          (4): Conv1d(16, 16, kernel_size=(2,), stride=(1,), padding=(1,))
          (5): Chomp1d()
          (6): ReLU()
          (7): Dropout(p=0.2, inplace=False)
        )
        (downsample): Conv1d(5, 16, kernel_size=(1,), stride=(1,))
        (relu): ReLU()
      )
      (1): TemporalBlock(
        (conv1): Conv1d(16, 32, kernel_size=(2,

Prepare Training Dataset Indo

In [ ]:
promo_ids = list(indo_pd_df["PromoIDText"].unique())

len(promo_ids)

20129

In [ ]:
indo_multied = indo_pd_df.set_index(["PromoIDText", "ProductNameSku_PPH", "Customer", "WeekSkID"])

In [ ]:
indo_multied

PromoGroup  ...  ActualBaselineVolume
PromoIDText ProductNameSku_PPH                   Customer          WeekSkID                 ...                      
P-00052548  BUAVITA ORANGE NEW 24X245ML          SD SMALL          2021.25   Not Available  ...             236.00000
            BUAVITA APPLE NEW 24X245ML           SD SMALL          2021.26   Not Available  ...             108.00000
            BUAVITA GUAVA RL WD 24X245ML         SD SMALL          2021.26   Not Available  ...            2364.00000
            BUAVITA LYCHEE NEW 24X245ML          SD SMALL          2021.26   Not Available  ...             111.00000
            BUAVITA MANGO NEW 24X245ML           SD SMALL          2021.26   Not Available  ...             148.00000
...                                                                                    ...  ...                   ...
P-00553233  LIFEBUOY BW MILD CARE PCH 12X825ML   TTB SECOND DEALER 2023.51   TradePromoCPP  ...              53.94286
            LIFEBUOY BW TOTAL 10 PCH 12X825ML    TTB SECOND DEALER 2023.51   TradePromoCPP  ...              74.97143
            LIFEBUOY BW LEMON FRESH PCH 12X825ML TTB SECOND DEALER 2023.52   TradePromoCPP  ...             105.14286
            LIFEBUOY BW MILD CARE PCH 12X825ML   TTB SECOND DEALER 2023.52   TradePromoCPP  ...              53.94286
            LIFEBUOY BW TOTAL 10 PCH 12X825ML    TTB SECOND DEALER 2023.52   TradePromoCPP  ...              74.97143

[460869 rows x 44 columns]

In [ ]:
indo_multied.loc["P-00052548", "BUAVITA ORANGE NEW 24X245ML", "SD SMALL"]

/root/.ipykernel/1307/command-2107240497971665-1440001807:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  indo_multied.loc["P-00052548", "BUAVITA ORANGE NEW 24X245ML", "SD SMALL"]


,PromoGroup,InStoreStartWeek,InStoreEndWeek,PromoFlag,PromotionStatus,InstoreStartDate,InstoreEndDate,ShipmentStartDate,ShipmentEndDate,PromoMechanic,PromoShopperMechanic,Level1Name_PPH,Category,Level4Name_PPH,Brand,Level6Name_PPH,CPG,SPF,fundtype,ListPrice,PlannedPromoSalesValueSellIn,PlannedPromoSalesVolumeSellIn,PlannedNetPromoGSVSellIn,PlannedNetPromoNIVSellIn,PlannedTTSOnSpend,PlannedNetPromoTOSellIn,PlannedTTSOffSpend,PlannedNetPromoGrossProfitsSellIn,PlannedNetPromoCOGSSellIn,PlannedNetPromoPBOSellIn,PlannedBaselineValue,PlannedBaselineVolume,PlannedBaseGSVSellIn,PlannedBaseTTSOnSpend,PlannedBaseNIVSellIn,PlannedBaseTTSOffSpend,PlannedBaseTOSellIn,PlannedBaseGrossProfitsSellIn,PlannedBaseCOGSSellIn,PlannedBasePBOSellIn,PlannedPromoSpend,ActualNetPromoSalesVolumeSellOut,ActualBaselineValue,ActualBaselineVolume
WeekSkID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021.25,Not Available,2021.23,2021.26,BUAVITA 250ML 6-12%/SD DOLPHIN/JUICE/CCD/GT/WE...,Complete,2021-06-11,2021-06-30,2021-06-11,2021-06-30,TIERED UNIT - DISCOUNT %,TIERED UNIT - DISCOUNT %,NUTRITION & ICE CREAM,SOY & FRUIT BEVERAGES,JUICE,BUAVITA,BUAVITA ORANGE,BUAVITA FRUIT JUICES,BUAVITA ORANGE NEW 24X245ML,BBTCPP,5409.09,2394346.555,442.65238,2394346.555,2258922.298,135424.2578,2045051.979,213870.319,863841.0344,1181210.944,863841.0344,1370103.179,253.29643,1370103.179,77493.03579,1292610.143,18253.94296,1274356.2,598438.6431,675917.5568,598438.6431,0,312.47223,1382022.28,236.0
2021.26,Not Available,2021.23,2021.26,BUAVITA 250ML 6-12%/SD DOLPHIN/JUICE/CCD/GT/WE...,Complete,2021-06-11,2021-06-30,2021-06-11,2021-06-30,TIERED UNIT - DISCOUNT %,TIERED UNIT - DISCOUNT %,NUTRITION & ICE CREAM,SOY & FRUIT BEVERAGES,JUICE,BUAVITA,BUAVITA ORANGE,BUAVITA FRUIT JUICES,BUAVITA ORANGE NEW 24X245ML,BBTCPP,5409.09,2394346.555,442.65238,2394346.555,2258922.298,135424.2578,2045051.979,213870.319,863841.0344,1181210.944,863841.0344,1370103.179,253.29643,1370103.179,77493.03579,1292610.143,18253.94296,1274356.2,598438.6431,675917.5568,598438.6431,0,13.68250,503044.44,93.0
2021.23,Not Available,2021.23,2021.26,BUAVITA 250ML 6-12%/SD DOLPHIN/JUICE/CCD/GT/WE...,Complete,2021-06-11,2021-06-30,2021-06-11,2021-06-30,TIERED UNIT - DISCOUNT %,TIERED UNIT - DISCOUNT %,NUTRITION & ICE CREAM,SOY & FRUIT BEVERAGES,JUICE,BUAVITA,BUAVITA ORANGE,BUAVITA FRUIT JUICES,BUAVITA ORANGE NEW 24X245ML,BBTCPP,5409.09,2394346.555,442.65238,2394346.555,2258922.298,135424.2578,2045051.979,213870.319,863841.0344,1181210.944,863841.0344,1370103.179,253.29643,1370103.179,77493.03579,1292610.143,18253.94296,1274356.2,598438.6431,675917.5568,598438.6431,0,74.14056,319135.72,59.0
2021.24,Not Available,2021.23,2021.26,BUAVITA 250ML 6-12%/SD DOLPHIN/JUICE/CCD/GT/WE...,Complete,2021-06-11,2021-06-30,2021-06-11,2021-06-30,TIERED UNIT - DISCOUNT %,TIERED UNIT - DISCOUNT %,NUTRITION & ICE CREAM,SOY & FRUIT BEVERAGES,JUICE,BUAVITA,BUAVITA ORANGE,BUAVITA FRUIT JUICES,BUAVITA ORANGE NEW 24X245ML,BBTCPP,5409.09,2394346.555,442.65238,2394346.555,2258922.298,135424.2578,2045051.979,213870.319,863841.0344,1181210.944,863841.0344,1370103.179,253.29643,1370103.179,77493.03579,1292610.143,18253.94296,1274356.2,598438.6431,675917.5568,598438.6431,0,56.40658,741043.96,137.0
